# ConveyorGuard - 03. ML Baseline 비교

**Traditional ML 모델 + 간이 베이스라인 비교**

- **Accelerator: GPU T4 x2** (ML은 CPU 사용, GPU는 DL 비교용)
- 입력: `train_data.joblib`, `val_data.joblib`, `test_data.joblib`
- 출력: `ml_comparison_results.csv`, 각 모델 `.pkl`

---

### Pipeline
> `00_eda` → `01_preprocess` → `02` / **`03_ml_baseline`**(병렬) → `04_dl_tuning` → `05_llm_comparison`

| 입력 데이터셋 | 출력 | Accelerator |
|--------------|------|-------------|
| `conveyorguard-preprocess` (01 출력) | `ml_comparison_results.csv` | GPU T4 x2 |

## 1. 환경 설정

In [1]:
# 패키지 설치
!pip install xgboost lightgbm catboost -q

import os
import sys
import numpy as np
import pandas as pd
import joblib
from pathlib import Path
import time
import warnings
import gc
warnings.filterwarnings('ignore')

# Plotly 설정
import plotly.io as pio
pio.renderers.default = 'iframe'

# ML 라이브러리
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

# 환경 정보
print(f"Python: {sys.version}")
try:
    import torch
    print(f"PyTorch: {torch.__version__}")
    print(f"CUDA: {torch.version.cuda}")
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            props = torch.cuda.get_device_properties(i)
            print(f"GPU {i}: {props.name} ({getattr(props, 'total_memory', None) or getattr(props, 'total_mem', 0) / 1024**3:.1f} GB)")
except ImportError:
    print("PyTorch: not installed")

print(f"\n✅ 라이브러리 로드 완료")

# 전체 실행 시간 측정 시작
_notebook_start = time.time()

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch: 2.8.0+cu126
CUDA: 12.6
GPU 0: Tesla T4 (15637086208.0 GB)
GPU 1: Tesla T4 (15637086208.0 GB)

✅ 라이브러리 로드 완료


In [2]:
# 경로 설정
DATA_DIR = Path('/kaggle/input/conveyorguard-preprocess')
if not DATA_DIR.exists():
    DATA_DIR = Path('/kaggle/input/preprocess')
    if not DATA_DIR.exists():
        print("⚠️ 데이터셋을 추가해주세요!")
        print("   Add Data → Your Datasets → 전처리된 데이터")
    else:
        print(f"📂 데이터 경로: {DATA_DIR}")
else:
    print(f"📂 데이터 경로: {DATA_DIR}")

OUTPUT_DIR = Path('/kaggle/working')
print(f"📁 출력 폴더: {OUTPUT_DIR}")

📂 데이터 경로: /kaggle/input/conveyorguard-preprocess
📁 출력 폴더: /kaggle/working


## 2. 데이터 로드

In [3]:
# joblib 로드
print("📂 데이터 로드 중...")

train_data = joblib.load(DATA_DIR / 'train_data.joblib')
val_data = joblib.load(DATA_DIR / 'val_data.joblib')
test_data = joblib.load(DATA_DIR / 'test_data.joblib')

print(f"\n📊 데이터 크기:")
print(f"   Train: {train_data['sensors'].shape[0]:,}개")
print(f"   Val: {val_data['sensors'].shape[0]:,}개")
print(f"   Test: {test_data['sensors'].shape[0]:,}개")

# 클래스 분포
print(f"\n📈 클래스 분포 (Train):")
unique, counts = np.unique(train_data['labels'], return_counts=True)
for u, c in zip(unique, counts):
    label_names = ['정상', '경미', '중간', '심각']
    print(f"   {label_names[u]}: {c:,} ({c/len(train_data['labels'])*100:.1f}%)")

📂 데이터 로드 중...

📊 데이터 크기:
   Train: 7,311개
   Val: 1,554개
   Test: 1,608개

📈 클래스 분포 (Train):
   정상: 3,586 (49.0%)
   경미: 1,682 (23.0%)
   중간: 1,642 (22.5%)
   심각: 401 (5.5%)


## 3. 피처 엔지니어링

ML 모델은 이미지를 직접 처리 못하므로 통계값으로 변환

In [4]:
def extract_features(data: dict) -> np.ndarray:
    """
    멀티모달 데이터를 ML용 피처로 변환
    
    Input:
        sensors: (N, 30, 8)   → flatten + 통계
        images: (N, 30, 60, 80) → 통계값
        external: (N, 30, 3)  → flatten + 통계
    
    Output:
        features: (N, num_features)
    """
    N = data['sensors'].shape[0]
    features_list = []
    
    # 1. 센서 피처 (8개 센서 × 30 타임스텝)
    sensors = data['sensors']  # (N, 30, 8)
    
    # 센서별 통계 (mean, std, max, min, last)
    sensor_mean = sensors.mean(axis=1)  # (N, 8)
    sensor_std = sensors.std(axis=1)    # (N, 8)
    sensor_max = sensors.max(axis=1)    # (N, 8)
    sensor_min = sensors.min(axis=1)    # (N, 8)
    sensor_last = sensors[:, -1, :]     # (N, 8) - 마지막 값
    sensor_diff = sensors[:, -1, :] - sensors[:, 0, :]  # (N, 8) - 변화량
    
    features_list.extend([sensor_mean, sensor_std, sensor_max, sensor_min, sensor_last, sensor_diff])
    
    # 2. 이미지 피처 (열화상 → 통계값)
    images = data['images']  # (N, 30, 60, 80)
    
    # 프레임별 통계 후 시계열 통계
    img_frame_mean = images.mean(axis=(2, 3))  # (N, 30)
    img_frame_max = images.max(axis=(2, 3))    # (N, 30)
    img_frame_std = images.std(axis=(2, 3))    # (N, 30)
    
    # 시계열 통계
    img_mean = img_frame_mean.mean(axis=1, keepdims=True)  # (N, 1)
    img_std = img_frame_mean.std(axis=1, keepdims=True)    # (N, 1)
    img_max = img_frame_max.max(axis=1, keepdims=True)     # (N, 1)
    img_max_mean = img_frame_max.mean(axis=1, keepdims=True)  # (N, 1)
    img_last_mean = img_frame_mean[:, -1:]   # (N, 1)
    img_last_max = img_frame_max[:, -1:]     # (N, 1)
    img_trend = img_frame_mean[:, -1:] - img_frame_mean[:, 0:1]  # (N, 1) - 온도 변화 추세
    
    features_list.extend([img_mean, img_std, img_max, img_max_mean, img_last_mean, img_last_max, img_trend])
    
    # 3. 외부환경 피처
    external = data['external']  # (N, 30, 3)
    
    ext_mean = external.mean(axis=1)  # (N, 3)
    ext_std = external.std(axis=1)    # (N, 3)
    ext_last = external[:, -1, :]     # (N, 3)
    
    features_list.extend([ext_mean, ext_std, ext_last])
    
    # 모든 피처 결합
    features = np.concatenate(features_list, axis=1)
    
    return features

print("✅ 피처 추출 함수 정의 완료")

✅ 피처 추출 함수 정의 완료


In [5]:
# 피처 추출
print("🔧 피처 추출 중...")

X_train = extract_features(train_data)
X_val = extract_features(val_data)
X_test = extract_features(test_data)

y_train = train_data['labels']
y_val = val_data['labels']
y_test = test_data['labels']

print(f"\n📊 피처 shape:")
print(f"   X_train: {X_train.shape}")
print(f"   X_val: {X_val.shape}")
print(f"   X_test: {X_test.shape}")
print(f"\n   총 피처 수: {X_train.shape[1]}개")

# 메모리 정리
del train_data, val_data, test_data
gc.collect()

🔧 피처 추출 중...

📊 피처 shape:
   X_train: (7311, 64)
   X_val: (1554, 64)
   X_test: (1608, 64)

   총 피처 수: 64개


37

In [6]:
# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# 스케일러 저장
joblib.dump(scaler, OUTPUT_DIR / 'ml_scaler.pkl')
print("✅ 스케일링 완료 & 저장")

✅ 스케일링 완료 & 저장


## 4. ML 모델 학습 & 평가

In [7]:
# 결과 저장용
results = []

def train_and_evaluate(name, model, X_train, y_train, X_val, y_val, X_test, y_test):
    """모델 학습 및 평가"""
    print(f"\n{'='*60}")
    print(f"🚀 {name} 학습 중...")
    
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # 예측
    val_pred = model.predict(X_val)
    test_pred = model.predict(X_test)
    
    # 평가
    val_acc = accuracy_score(y_val, val_pred) * 100
    test_acc = accuracy_score(y_test, test_pred) * 100
    val_f1 = f1_score(y_val, val_pred, average='macro') * 100
    test_f1 = f1_score(y_test, test_pred, average='macro') * 100
    
    print(f"   ⏱️ 학습 시간: {train_time:.1f}초")
    print(f"   📊 Val  - Acc: {val_acc:.2f}%, F1: {val_f1:.2f}%")
    print(f"   📊 Test - Acc: {test_acc:.2f}%, F1: {test_f1:.2f}%")
    
    # 결과 저장
    results.append({
        'Model': name,
        'Val_Acc': round(val_acc, 2),
        'Test_Acc': round(test_acc, 2),
        'Val_F1': round(val_f1, 2),
        'Test_F1': round(test_f1, 2),
        'Train_Time': round(train_time, 1)
    })
    
    return model, test_pred

print("✅ 평가 함수 정의 완료")

✅ 평가 함수 정의 완료


### 4.0 간이 베이스라인 (4종)

빠른 비교를 위한 간단한 ML 모델

In [8]:
# === 간이 베이스라인 4종 ===
simple_models = {
    'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1),
    'DecisionTree': DecisionTreeClassifier(max_depth=20, random_state=42),
    'KNN (k=5)': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
    'SVM (RBF)': SVC(kernel='rbf', random_state=42)
}

for name, model in simple_models.items():
    train_and_evaluate(name, model, X_train, y_train, X_val, y_val, X_test, y_test)

print(f"\n✅ 간이 베이스라인 4종 완료")


🚀 LogisticRegression 학습 중...
   ⏱️ 학습 시간: 2.0초
   📊 Val  - Acc: 86.42%, F1: 86.08%
   📊 Test - Acc: 87.31%, F1: 86.81%

🚀 DecisionTree 학습 중...
   ⏱️ 학습 시간: 0.6초
   📊 Val  - Acc: 93.44%, F1: 93.21%
   📊 Test - Acc: 92.97%, F1: 92.00%

🚀 KNN (k=5) 학습 중...
   ⏱️ 학습 시간: 0.0초
   📊 Val  - Acc: 89.90%, F1: 90.20%
   📊 Test - Acc: 89.12%, F1: 88.28%

🚀 SVM (RBF) 학습 중...
   ⏱️ 학습 시간: 1.0초
   📊 Val  - Acc: 86.87%, F1: 87.07%
   📊 Test - Acc: 87.75%, F1: 87.01%

✅ 간이 베이스라인 4종 완료


### 4.1 XGBoost

In [9]:
xgb_model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    tree_method='hist'
)

xgb_model, xgb_pred = train_and_evaluate(
    'XGBoost', xgb_model, 
    X_train_scaled, y_train, 
    X_val_scaled, y_val, 
    X_test_scaled, y_test
)

joblib.dump(xgb_model, OUTPUT_DIR / 'xgboost_model.pkl')


🚀 XGBoost 학습 중...
   ⏱️ 학습 시간: 3.8초
   📊 Val  - Acc: 97.04%, F1: 96.98%
   📊 Test - Acc: 96.70%, F1: 96.28%


['/kaggle/working/xgboost_model.pkl']

### 4.2 LightGBM

In [10]:
lgb_model = lgb.LGBMClassifier(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

lgb_model, lgb_pred = train_and_evaluate(
    'LightGBM', lgb_model,
    X_train_scaled, y_train,
    X_val_scaled, y_val,
    X_test_scaled, y_test
)

joblib.dump(lgb_model, OUTPUT_DIR / 'lightgbm_model.pkl')


🚀 LightGBM 학습 중...
   ⏱️ 학습 시간: 2.7초
   📊 Val  - Acc: 96.98%, F1: 96.69%
   📊 Test - Acc: 96.89%, F1: 96.43%


['/kaggle/working/lightgbm_model.pkl']

### 4.3 RandomForest

In [11]:
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_model, rf_pred = train_and_evaluate(
    'RandomForest', rf_model,
    X_train_scaled, y_train,
    X_val_scaled, y_val,
    X_test_scaled, y_test
)

joblib.dump(rf_model, OUTPUT_DIR / 'randomforest_model.pkl')


🚀 RandomForest 학습 중...
   ⏱️ 학습 시간: 3.7초
   📊 Val  - Acc: 96.53%, F1: 96.36%
   📊 Test - Acc: 95.58%, F1: 95.19%


['/kaggle/working/randomforest_model.pkl']

### 4.4 CatBoost

In [12]:
cat_model = CatBoostClassifier(
    iterations=300,
    depth=8,
    learning_rate=0.1,
    random_state=42,
    verbose=False,
    thread_count=-1
)

cat_model, cat_pred = train_and_evaluate(
    'CatBoost', cat_model,
    X_train_scaled, y_train,
    X_val_scaled, y_val,
    X_test_scaled, y_test
)

joblib.dump(cat_model, OUTPUT_DIR / 'catboost_model.pkl')


🚀 CatBoost 학습 중...
   ⏱️ 학습 시간: 21.5초
   📊 Val  - Acc: 96.59%, F1: 96.37%
   📊 Test - Acc: 96.46%, F1: 96.01%


['/kaggle/working/catboost_model.pkl']

## 5. 결과 비교

In [13]:
# CNN+Transformer 결과 추가 (02에서 얻은 결과)
# ⚠️ 실제 02 실행 후 이 값들을 업데이트하세요!
results.append({
    'Model': 'CNN+Transformer (DL)',
    'Val_Acc': 93.24,   
    'Test_Acc': 92.72,  
    'Val_F1': 93.0,
    'Test_F1': 93.0,
    'Train_Time': 2178  # 36.3분
})

# DataFrame 생성
results_df = pd.DataFrame(results)

# Test_Acc 기준 정렬
results_df = results_df.sort_values('Test_Acc', ascending=False).reset_index(drop=True)

print("\n" + "="*70)
print("📊 모델 성능 비교")
print("="*70)
print(results_df.to_string(index=False))

# CSV 저장
results_df.to_csv(OUTPUT_DIR / 'ml_comparison_results.csv', index=False)
print(f"\n💾 결과 저장: ml_comparison_results.csv")


📊 모델 성능 비교
               Model  Val_Acc  Test_Acc  Val_F1  Test_F1  Train_Time
            LightGBM    96.98     96.89   96.69    96.43         2.7
             XGBoost    97.04     96.70   96.98    96.28         3.8
            CatBoost    96.59     96.46   96.37    96.01        21.5
        RandomForest    96.53     95.58   96.36    95.19         3.7
        DecisionTree    93.44     92.97   93.21    92.00         0.6
CNN+Transformer (DL)    93.24     92.72   93.00    93.00      2178.0
           KNN (k=5)    89.90     89.12   90.20    88.28         0.0
           SVM (RBF)    86.87     87.75   87.07    87.01         1.0
  LogisticRegression    86.42     87.31   86.08    86.81         2.0

💾 결과 저장: ml_comparison_results.csv


In [14]:
import plotly
print(f'Plotly: {plotly.__version__}')

Plotly: 5.24.1


In [15]:
# 시각화 (Plotly) - Kaggle용
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Kaggle 렌더러 설정
pio.renderers.default = 'iframe'

# 색상 설정
colors = ['#2ecc71' if 'DL' in m or 'CNN' in m else '#3498db' for m in results_df['Model']]

# 서브플롯 생성
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Model Comparison - Test Accuracy', 'Model Comparison - Training Time'),
    horizontal_spacing=0.15
)

# 1. Accuracy 비교
fig.add_trace(
    go.Bar(
        y=results_df['Model'],
        x=results_df['Test_Acc'],
        orientation='h',
        marker_color=colors,
        text=[f'{v:.1f}%' for v in results_df['Test_Acc']],
        textposition='outside',
        name='Accuracy'
    ),
    row=1, col=1
)

# 2. 학습 시간 비교
time_text = [f'{v/60:.1f}분' if v > 60 else f'{v:.1f}초' for v in results_df['Train_Time']]
fig.add_trace(
    go.Bar(
        y=results_df['Model'],
        x=results_df['Train_Time'],
        orientation='h',
        marker_color=colors,
        text=time_text,
        textposition='outside',
        name='Train Time'
    ),
    row=1, col=2
)

# 레이아웃 설정
fig.update_xaxes(range=[70, 100], title_text='Test Accuracy (%)', row=1, col=1)
fig.update_xaxes(type='log', title_text='Training Time (seconds)', row=1, col=2)
fig.update_layout(
    height=400,
    width=1000,
    showlegend=False,
    title_text='ML Model Comparison',
    title_x=0.5
)

# 저장 & 표시
fig.write_html(OUTPUT_DIR / 'ml_comparison.html')
fig.show(renderer='iframe')

print(f"\n📊 시각화 저장: ml_comparison.html")


📊 시각화 저장: ml_comparison.html


## 6. Best ML 모델 상세 분석

In [16]:
# Best ML 모델 찾기 (DL 제외)
ml_only = results_df[~results_df['Model'].str.contains('DL|CNN')]
best_ml = ml_only.loc[ml_only['Test_Acc'].idxmax(), 'Model']

print(f"\n🏆 Best ML Model: {best_ml}")

# Best 모델의 예측 결과
if 'XGBoost' in best_ml:
    best_pred = xgb_pred
elif 'LightGBM' in best_ml:
    best_pred = lgb_pred
elif 'RandomForest' in best_ml:
    best_pred = rf_pred
else:
    best_pred = cat_pred

# Classification Report
label_names = ['정상', '경미', '중간', '심각']
print(f"\n📋 Classification Report ({best_ml}):")
print(classification_report(y_test, best_pred, target_names=label_names))


🏆 Best ML Model: LightGBM

📋 Classification Report (LightGBM):
              precision    recall  f1-score   support

          정상       0.99      0.99      0.99       788
          경미       0.94      0.94      0.94       371
          중간       0.96      0.95      0.95       361
          심각       0.98      0.98      0.98        88

    accuracy                           0.97      1608
   macro avg       0.96      0.96      0.96      1608
weighted avg       0.97      0.97      0.97      1608



In [17]:
# Confusion Matrix (Plotly)
import plotly.figure_factory as ff
import plotly.io as pio

pio.renderers.default = 'iframe'

cm = confusion_matrix(y_test, best_pred)

# 비율 계산 (annotation용)
cm_text = [[f'{val}\n({val/cm.sum()*100:.1f}%)' for val in row] for row in cm]

fig = ff.create_annotated_heatmap(
    z=cm,
    x=label_names,
    y=label_names,
    annotation_text=cm_text,
    colorscale='Blues',
    showscale=True
)

fig.update_layout(
    title=f'Confusion Matrix - {best_ml}',
    xaxis_title='Predicted',
    yaxis_title='Actual',
    width=600,
    height=500
)

# y축 뒤집기 (sklearn 스타일)
fig.update_yaxes(autorange='reversed')

fig.write_html(OUTPUT_DIR / 'ml_confusion_matrix.html')
fig.show(renderer='iframe')

print(f"\n📊 ml_confusion_matrix.html 저장 완료")


📊 ml_confusion_matrix.html 저장 완료


## 7. Feature Importance

In [18]:
# Feature names 생성
feature_names = []

# 센서 피처 이름
sensor_names = ['NTC', 'PM1.0', 'PM2.5', 'PM10', 'CT1', 'CT2', 'CT3', 'CT4']
for stat in ['mean', 'std', 'max', 'min', 'last', 'diff']:
    for s in sensor_names:
        feature_names.append(f'sensor_{s}_{stat}')

# 이미지 피처 이름
for name in ['img_mean', 'img_std', 'img_max', 'img_max_mean', 'img_last_mean', 'img_last_max', 'img_trend']:
    feature_names.append(name)

# 외부환경 피처 이름
ext_names = ['temp', 'humidity', 'illuminance']
for stat in ['mean', 'std', 'last']:
    for e in ext_names:
        feature_names.append(f'ext_{e}_{stat}')

print(f"총 피처 수: {len(feature_names)}")

총 피처 수: 64


In [19]:
# XGBoost Feature Importance (Plotly)
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'iframe'

if hasattr(xgb_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': feature_names[:len(xgb_model.feature_importances_)],
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=True)  # ascending=True로 변경 (plotly barh용)
    
    # Top 20
    top_20 = importance_df.tail(20)  # tail로 변경
    
    fig = go.Figure(go.Bar(
        x=top_20['importance'],
        y=top_20['feature'],
        orientation='h',
        marker_color='#3498db',
        text=[f'{v:.4f}' for v in top_20['importance']],
        textposition='outside'
    ))
    
    fig.update_layout(
        title='Top 20 Feature Importance (XGBoost)',
        xaxis_title='Importance',
        yaxis_title='Feature',
        height=600,
        width=800,
        margin=dict(l=200)  # 왼쪽 여백 (긴 피처명용)
    )
    
    fig.write_html(OUTPUT_DIR / 'feature_importance.html')
    fig.show(renderer='iframe')
    
    print(f"\n📊 feature_importance.html 저장 완료")
    
    # 중요 피처 저장
    importance_df.sort_values('importance', ascending=False).to_csv(
        OUTPUT_DIR / 'feature_importance.csv', index=False
    )


📊 feature_importance.html 저장 완료


## 8. 출력 파일 확인

In [20]:
# 출력 파일 목록
print("\n📦 생성된 파일:")
for f in sorted(OUTPUT_DIR.glob('*')):
    size_mb = f.stat().st_size / (1024**2)
    print(f"   {f.name}: {size_mb:.2f} MB")


📦 생성된 파일:
   .virtual_documents: 0.00 MB
   catboost_info: 0.00 MB
   catboost_model.pkl: 2.97 MB
   feature_importance.csv: 0.00 MB
   feature_importance.html: 4.36 MB
   iframe_figures: 0.00 MB
   lightgbm_model.pkl: 3.13 MB
   ml_comparison.html: 4.36 MB
   ml_comparison_results.csv: 0.00 MB
   ml_confusion_matrix.html: 4.36 MB
   ml_scaler.pkl: 0.00 MB
   randomforest_model.pkl: 11.64 MB
   xgboost_model.pkl: 2.11 MB


## Kaggle 데이터셋 업로드 (API)

In [21]:
import os, json

os.environ['KAGGLE_USERNAME'] = 'kukass'
os.environ['KAGGLE_KEY'] = 'KGAT_c973fff8eb3e1ccb19f3e9d683eb17dc'

UPLOAD_DIR = '/kaggle/working/dataset_upload'
os.makedirs(UPLOAD_DIR, exist_ok=True)

output_files = ['ml_scaler.pkl', 'xgboost_model.pkl', 'lightgbm_model.pkl', 'randomforest_model.pkl', 'catboost_model.pkl', 'ml_comparison_results.csv', 'ml_comparison.html', 'ml_confusion_matrix.html', 'feature_importance.html', 'feature_importance.csv']
for f in output_files:
    src = f'/kaggle/working/{f}'
    dst = f'{UPLOAD_DIR}/{f}'
    if os.path.exists(src) and not os.path.exists(dst):
        os.symlink(src, dst)

meta = {
    "title": "conveyorguard-ml",
    "id": "kukass/conveyorguard-ml",
    "licenses": [{"name": "CC0-1.0"}]
}
with open(f'{UPLOAD_DIR}/dataset-metadata.json', 'w') as f:
    json.dump(meta, f)

!kaggle datasets create -p {UPLOAD_DIR} --dir-mode zip

print("\n✅ conveyorguard-ml 데이터셋 업로드 완료!")
print("   → 06_ensemble, 07_final_comparison에서 사용 가능")

Starting upload for file feature_importance.html
100%|██████████████████████████████████████| 4.36M/4.36M [00:00<00:00, 9.14MB/s]
Upload successful: feature_importance.html (4MB)
Starting upload for file randomforest_model.pkl
100%|██████████████████████████████████████| 11.6M/11.6M [00:00<00:00, 24.6MB/s]
Upload successful: randomforest_model.pkl (12MB)
Starting upload for file ml_comparison.html
100%|██████████████████████████████████████| 4.36M/4.36M [00:00<00:00, 9.17MB/s]
Upload successful: ml_comparison.html (4MB)
Starting upload for file lightgbm_model.pkl
100%|██████████████████████████████████████| 3.13M/3.13M [00:00<00:00, 7.06MB/s]
Upload successful: lightgbm_model.pkl (3MB)
Starting upload for file ml_scaler.pkl
100%|██████████████████████████████████████| 2.10k/2.10k [00:00<00:00, 5.33kB/s]
Upload successful: ml_scaler.pkl (2KB)
Starting upload for file catboost_model.pkl
100%|██████████████████████████████████████| 2.97M/2.97M [00:00<00:00, 6.66MB/s]
Upload successful: ca

---

## 📋 결과 요약

### ML vs DL 비교

| 모델 유형 | 장점 | 단점 | 적합한 상황 |
|-----------|------|------|-------------|
| **간이 ML** | 초고속, 해석 용이 | 정확도 낮음 | 빠른 POC, 비교 기준선 |
| **앙상블 ML (XGBoost 등)** | 빠름, 해석 가능, 가벼움 | 이미지 직접 처리 불가 | 센서 위주 분석, 빠른 추론 |
| **DL (CNN+Transformer)** | 이미지 직접 처리, 높은 정확도 | 학습 시간 김, GPU 필요 | 멀티모달 분석, 최고 성능 필요 |

### 다음 단계
- `04_dl_tuning.ipynb`: Optuna 하이퍼파라미터 최적화로 DL 성능 개선
- `05_llm_comparison.ipynb`: LLM 진단 비교